In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import collections
from dataclasses import dataclass, field 

sc = SparkContext.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1675148691692_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# RDD: mapValue and reduceByKey function

In [5]:
lines = sc.textFile("s3://370784835428-datalake/udemy/u.data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
ratings = lines.map(lambda x: x.split()[2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
result = ratings.countByValue()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
sortedResults = collections.OrderedDict(sorted(result.items()))
for key, value in sortedResults.items():
    print("%s %i" % (key, value))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1 6110
2 11370
3 27145
4 34174
5 21201

In [13]:
def parseLine(line):
    fields = line.split(',')
    age = int(fields[2])
    numFriends = int(fields[3])
    return (age, numFriends)

lines = sc.textFile("s3://370784835428-datalake/udemy/SparkCourse/SparkCourse/fakefriends.csv")
rdd = lines.map(parseLine) 

totalsByAge = rdd.mapValues(lambda x:(x, 1)).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) 

totalsByAgeCollected = totalsByAge.collect() 

averagesByAge = totalsByAge.mapValues(lambda x:x[0]/x[1]) 

results = averagesByAge.collect()
  
for result in results:
   print(result)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(26, 242.05882352941177)
(40, 250.8235294117647)
(68, 269.6)
(54, 278.0769230769231)
(38, 193.53333333333333)
(56, 306.6666666666667)
(36, 246.6)
(22, 206.42857142857142)
(60, 202.71428571428572)
(30, 235.8181818181818)
(42, 303.5)
(48, 281.4)
(50, 254.6)
(32, 207.9090909090909)
(58, 116.54545454545455)
(64, 281.3333333333333)
(52, 340.6363636363636)
(24, 233.8)
(20, 165.0)
(62, 220.76923076923077)
(44, 282.1666666666667)
(28, 209.1)
(66, 276.44444444444446)
(46, 223.69230769230768)
(18, 343.375)
(34, 245.5)
(33, 325.3333333333333)
(55, 295.53846153846155)
(59, 220.0)
(37, 249.33333333333334)
(27, 228.125)
(53, 222.85714285714286)
(57, 258.8333333333333)
(43, 230.57142857142858)
(35, 211.625)
(45, 309.53846153846155)
(67, 214.625)
(19, 213.27272727272728)
(51, 302.14285714285717)
(25, 197.45454545454547)
(21, 350.875)
(49, 184.66666666666666)
(39, 169.28571428571428)
(31, 267.25)
(41, 268.55555555555554)
(69, 235.2)
(65, 298.2)
(61, 256.22222222222223)
(29, 215.91666666666666)
(47, 233

# RDD: filter function

def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)

lines = sc.textFile("s3://thanhtt-0000-datalake/udemy/SparkCourse/1800.csv")
parsedLines = lines.map(parseLine)
minTemps = parsedLines.filter(lambda x: "TMIN" in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: min(x,y))
results = minTemps.collect();

for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))


# RDD: flatMap function

import re

def normalizeWords(text):
    return re.compile(r'\W+',re.UNICODE).split(text.lower())

input = sc.textFile("s3://thanhtt-0000-datalake/udemy/SparkCourse/book.txt")
words = input.flatMap(normalizeWords) # the text split into many rows

wordCounts = words.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
wordCountsSorted = wordCounts.map(lambda x: (x[0],x[1])).map(lambda x:(x[1],x[0]) ).sortByKey()
# for item in wordCountsSorted.collect():# comment because the list is too long
    # print(item)
print("Result is commented because it's too long")
results = wordCountsSorted.collect()

# for result in results:
#     count = str(result[0]) 
#     word = result[1].encode("ascii","ignore")
    
#     # comment because the list is too long
#     if(word):
#         print(cleanWord,count)


# **DataFrame: Spark SQL**

from pyspark.sql import Row

def mapper(line):
    fields = line.split(',')
    return Row(ID = int(fields[0]), name = str(fields[1].encode("utf-8")),
               age = int(fields[2]), numFriends = int(fields[3]))

lines = sc.textFile("s3://thanhtt-0000-datalake/udemy/SparkCourse/fakefriends.csv")
people = lines.map(mapper)

# Infer the schema, and register the DataFrame as table.
schemaPeople = spark.createDataFrame(people).cache()
schemaPeople.createOrReplaceTempView("people")

# SQL can be run over DataFrames that have been register as a table
teenagers = spark.sql("SELECT * FROM people WHERE age >=13 and age <=19")

# The results of SQL queries are RDDs and support all the normal RDD operations
for teen in teenagers.collect():
    print(teen)
    
# We can also use functions instead of SQL queries
schemaPeople.groupBy("age").count().orderBy("age").show()


# **DataFrame: Infer Schema and common function to work with**

from pyspark.sql import functions as func

people = spark.read.option("header","true").option("inferSchema","true")\
            .csv("s3://thanhtt-0000-datalake/udemy/SparkCourse/fakefriends-header.csv")

print("here is our inferred schema")
people.printSchema()

print("display the name column")
people.select("name").show()

print("filter out anyone over 21")
people.filter(people.age < 21).show()

print("group by age")
people.groupBy("age").count().show()

print("make everyone 10 year older")
people.select(people.name, people.age +10).show()

print("sorted")
friendByAge = people.select("age","friends")
friendByAge.groupBy("age").avg("friends").sort("age").show()

print("formatted more nicely")
friendByAge.groupBy("age").agg(func.round(func.avg("friends"),2)).sort("age").show()

print("with a custom column name")
friendByAge.groupBy("age").agg(func.round(func.avg("friends"),2).alias("friends_avg")).sort("age").show()


# **Word count with DataFrame(split unstructure text into multi row dataframe)**

# read each line of my book into a dataframe
inputDF = spark.read.text("s3://thanhtt-0000-datalake/udemy/SparkCourse/book.txt")
inputDF.show()

# split using a regular expression that extract words
words = inputDF.select(func.explode(func.split(inputDF.value,"\\W+")).alias("word"))
words.filter(words.word != "")

words.show()

# **DataFrame: StructType, StructField, IntegerType, FloatType**

In [4]:
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType

#create schema when reading customer order
customerOrderSchema = StructType([\
                                 StructField("cust_id", IntegerType(), True),
                                 StructField("item_id", IntegerType(), True),
                                 StructField("amount_spent", FloatType(), True)
                                 ])

#load up the data into spark dataset
customerDF = spark.read.schema(customerOrderSchema).csv("s3://370784835428-datalake/udemy/SparkCourse/SparkCourse/customer-orders.csv")

totalByCustomer = customerDF.groupBy("cust_id").agg(func.round(func.sum("amount_spent"),2).alias("total_spent"))

totalByCustomerSorted = totalByCustomer.sort("total_spent")

totalByCustomer.show(totalByCustomerSorted.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+
|cust_id|total_spent|
+-------+-----------+
|     65|    5140.35|
|     81|    5112.71|
|     76|    4904.21|
|     12|    4664.59|
|      1|     4958.6|
|      6|    5397.88|
|      3|    4659.63|
|     92|    5379.28|
|     64|    5288.69|
|     37|     4735.2|
|     61|    5497.48|
|     35|    5155.42|
|      4|    4815.05|
|      8|    5517.24|
|     87|     5206.4|
|     60|    5040.71|
|     90|    5290.41|
|     11|    5152.29|
|     33|    5254.66|
|     83|     4635.8|
|     99|    4172.29|
|     66|    4681.92|
|     74|    4647.13|
|     36|    4278.05|
|     31|    4765.05|
|     85|    5503.43|
|     53|     4945.3|
|     34|     5330.8|
|     44|    4756.89|
|     91|    4642.26|
|     93|    5265.75|
|     94|    4475.57|
|     57|     4628.4|
|     54|    6065.39|
|     19|    5059.43|
|     55|    5298.09|
|     59|    5642.89|
|     23|    4042.65|
|     51|    4975.22|
|     82|    4812.49|
|     70|    5368.25|
|     95|    4876.84|
|     21| 